# Test migration

In [ ]:
import os

os.chdir("../../")

In [ ]:
import time
from subprocess import run
from typing import Optional

import erdiagram
import sqlalchemy as sa
from lamin_logger import logger
from lndb._settings import InstanceSettings, settings
from lndb.test import get_package_name
from lndb.test._migrations_unit import (
    execute_model_definitions_match_ddl,
    get_migration_config,
    get_migration_id_from_scripts,
    migration_id_is_consistent,
)
from pytest_alembic.executor import CommandExecutor, ConnectionExecutor
from pytest_alembic.runner import MigrationContext
from sqlalchemy import MetaData, create_engine, func, select

from lnhub_rest import engine
from lnhub_rest.schema._core import SQLModel  # updated registry in it!
from lnhub_rest.schema.migrations.settings import PROD_URL
from lnhub_rest.schema.migrations.utils import include_name
from lndb.dev._clone import clone_test

from sqlalchemy import text

from lnhub_rest.schema.migrations.function._2023_02_21_a88f5298b8f7_v0_4_2 import (
    sql_functions,
)
from lnhub_rest.schema.migrations.rls._2023_02_21_a88f5298b8f7_v0_4_2 import (
    sql_rls_account,
    sql_rls_account_instance,
    sql_rls_instance,
    sql_rls_migration,
    sql_rls_storage,
    sql_rls_version,
)
from lnhub_rest.schema.migrations.rls._2023_03_09_0c4d4fe5f2c6_v0_6_1 import (
    sql_rls_migration as sql_rls_migration_2,
    sql_rls_version as sql_rls_version_2,
)
from lnhub_rest.schema.migrations.rls._2023_03_24_333fd693eac8_v0_6_1b import (
    sql_rls_owner_select_collaborators,
)

In [ ]:
package_name = get_package_name()
schema_package_loc = f"./{package_name}/schema"

In [ ]:
def get_migration_context(schema_package_loc):
    src_settings = InstanceSettings(
        storage_root="teststore",
        db=PROD_URL,
        name="testdb",
        owner="testuser1",
    )
    connection_string = clone_test(src_settings=src_settings, supabase=True)
    engine = sa.create_engine(connection_string, future=True)
    target_metadata = SQLModel.metadata
    config = get_migration_config(
        schema_package_loc,
        target_metadata=target_metadata,
        include_schemas=True,
        include_name=include_name,
    )
    command_executor = CommandExecutor.from_config(config)
    command_executor.configure(connection=engine)
    migration_context = MigrationContext.from_config(
        config, command_executor, ConnectionExecutor(), engine
    )
    # clone_test is currently only copying tables
    # so we need to create existing RLS and functions manually
    create_functions_and_rls(engine)
    return migration_context


def create_functions_and_rls(engine):
    with engine.connect() as conn:
        conn.execute(text(sql_functions))
        conn.execute(text(sql_rls_account))
        conn.execute(text(sql_rls_account_instance))
        conn.execute(text(sql_rls_instance))
        conn.execute(text(sql_rls_migration))
        conn.execute(text(sql_rls_storage))
        conn.execute(text(sql_rls_version))
        conn.execute(text(sql_rls_migration_2))
        conn.execute(text(sql_rls_version_2))
        # conn.execute(text(sql_rls_owner_select_collaborators))
        conn.commit()


def test_migration_id_is_consistent():
    assert migration_id_is_consistent(schema_package_loc)


def test_model_definitions_match_ddl_postgres():
    migration_context = get_migration_context(schema_package_loc)
    execute_model_definitions_match_ddl(migration_context)


def test_export_schema():
    migration_id = get_migration_id_from_scripts(schema_package_loc)
    metadata = sa.MetaData(bind=engine)
    metadata.reflect()
    graph = erdiagram.create_schema_graph(
        metadata=metadata,
        show_datatypes=False,
        show_indexes=False,
        rankdir="LR",
        concentrate=True,
    )
    try:
        graph.write_svg(f"./docs/_schemas/lnhub-schema-{migration_id}.svg")
    except FileNotFoundError:
        print("Did not write schema graph.")

In [ ]:
from lnhub_rest.schema.migrations.versions import *

In [ ]:
test_migration_id_is_consistent()

In [ ]:
test_model_definitions_match_ddl_postgres()

In [ ]:
test_export_schema()